In [27]:
import torch
import torch.nn as nn
import pickle
from sklearn.preprocessing import StandardScaler
import pandas as pd
from exp.exp_anomaly_detection_custom import Exp_Anomaly_Detection_custom

In [30]:
with open("./test_results/anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el2_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0/args.pkl", "rb") as f:
    args = pickle.load(f)

In [31]:
exp = Exp_Anomaly_Detection_custom(args)  
model = exp._build_model()

Use GPU: cuda:0


In [32]:
model.load_state_dict(torch.load("./checkpoints/anomaly_detection_custom_ANAM_TimesNet_custom_ftS_sl100_ll48_pl0_dm64_nh8_el2_dl1_df64_expand2_dc4_fc1_ebtimeF_dtTrue_test_0/checkpoint.pth"))

<All keys matched successfully>

In [24]:
print(args)

Namespace(activation='gelu', anomaly_ratio=1.0, augmentation_ratio=0, batch_size=128, c_out=1, channel_independence=1, checkpoints='./checkpoints/', d_conv=4, d_ff=64, d_layers=1, d_model=64, data='custom', data_path='Normal_uni.csv', dec_in=7, decomp_method='moving_avg', des='test', device=device(type='cuda', index=0), devices='0,1,2,3', discdtw=False, discsdtw=False, distil=True, down_sampling_layers=0, down_sampling_method=None, down_sampling_window=1, dropout=0.1, dtwwarp=False, e_layers=2, embed='timeF', enc_in=1, expand=2, extra_tag='', factor=1, features='S', freq='d', gpu=0, gpu_type='cuda', inverse=False, is_training=1, itr=1, jitter=False, label_len=48, learning_rate=0.0001, loss='MSE', lradj='type1', magwarp=False, mask_rate=0.25, model='TimesNet', model_id='ANAM', moving_avg=25, n_heads=8, num_kernels=6, num_workers=10, p_hidden_dims=[128, 128], p_hidden_layers=2, patch_len=16, patience=3, permutation=False, pred_len=0, randompermutation=False, root_path='./dataset/ANAM', r

In [ ]:
def predict(args, df, target, model) :
    scaler = StandardScaler()
    df_data = df[[target]]
    scaler.fit(df_data.values)
    data = scaler.transform(df_data.values)
    input_tensor = torch.tensor([data], dtype=torch.float32)
    anomaly_criterion = nn.MSELoss(reduce=False)

    with torch.no_grad():
        input_tensor = input_tensor.to(args.device)
        model = model.to(args.device)
        output = model(input_tensor, None, None, None)
        score = torch.mean(anomaly_criterion(input_tensor, output), dim=-1)
        score = score.detach().cpu().numpy()
        
    return score, output

In [200]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

def predict_dataframe(args, df, target, model, threshold=None):
    """
    args       : argparse avec seq_len, batch_size, device
    df         : DataFrame contenant au moins la colonne `target`
    target     : nom de la variable à analyser
    model      : modèle TimesNet déjà chargé
    threshold  : seuil optionnel pour flagger les anomalies
    """

    ###### 1. Normalisation 
    scaler = StandardScaler()
    data = scaler.fit_transform(df[[target]].values)

    ###### 2. Découpage en fenêtres
    seq_len = args.seq_len
    X = []
    i = 0

    while i <= len(data):
        if i + seq_len > len(data):
           X.append(data[len(data) - seq_len:len(data)]) 
        else:
            X.append(data[i:i+seq_len])
        i += seq_len  # On avance d'un pas de temps pour la prochaine fenêtre

    X = np.array(X)  # shape = (nb_seq, seq_len, 1)

    # ###### 3. Conversion en tenseur
    # X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)  # (nb_seq, seq_len, 1)

    ###### 4. Passage par batch dans le modèle 
    anomaly_criterion = nn.MSELoss(reduction="none")
    model = model.to(args.device)
    # model.eval()
    all_scores, all_recon = [], []

    with torch.no_grad():
        for i in range(0, len(X)):
            print(f"Processing batch {i+1}/{len(X)}")
            batch = torch.tensor([X[i]], dtype=torch.float32) # (1, seq_len, 1)
            batch = batch.to(args.device)  
            output = model(batch, None, None, None)  # reconstruction

            # Erreur
            loss = torch.mean(anomaly_criterion(batch, output), dim=-1).cpu().numpy()
            if i + 1 == len(X):
                all_scores.append(loss[0][seq_len - (len(data) % seq_len):].reshape(-1, 1))  # Last batch may not be full
            else:
                all_scores.append(loss[0].reshape(-1, 1))  # Reshape to (seq_len, 1)

            # Reconstruction → back to numpy
            recon = output.cpu().numpy()
            if i + 1 == len(X):
                print(f"Last batch, length: {len(data) % seq_len}")
                all_recon.append(recon[0][seq_len - (len(data)  % seq_len):])
            else:
                all_recon.append(recon[0])

    # Concatenate all scores and reconstructions
    all_scores = np.concatenate(all_scores, axis=0)  # (nb_seq, seq_len)
    all_recon = np.concatenate(all_recon, axis=0)

    all_recon = scaler.inverse_transform(all_recon)  # Inverse transform

    
    df_result = df.copy()
    df_result["reconstruction_error"] = all_scores
    df_result["reconstructed"] = all_recon

    if threshold is not None:
        df_result["is_anomaly"] = df_result["reconstruction_error"] > threshold

    return df_result


In [55]:
df_raw = pd.read_csv('./dataset/ANAM/Normal_uni.csv')
d = df_raw[:100]
d

,NAME,Cumul_precipitation,date
0,Bobo-Dioulasso,0.0,1/1/2019 0:00
1,Bobo-Dioulasso,0.0,1/2/2019 0:00
2,Bobo-Dioulasso,0.0,1/3/2019 0:00
3,Bobo-Dioulasso,0.0,1/4/2019 0:00
4,Bobo-Dioulasso,0.0,1/5/2019 0:00
...,...,...,...
95,Bobo-Dioulasso,0.0,4/6/2019 0:00
96,Bobo-Dioulasso,0.0,4/7/2019 0:00
97,Bobo-Dioulasso,0.0,4/8/2019 0:00
98,Bobo-Dioulasso,0.0,4/9/2019 0:00


,NAME,Cumul_precipitation,date
100,Bobo-Dioulasso,0.0,4/11/2019 0:00
101,Bobo-Dioulasso,0.0,4/12/2019 0:00


In [110]:
s, o = predict(args, d, 'Cumul_precipitation', model)

In [113]:
s

tensor([[3.8418e-03, 4.2544e-03, 1.4207e-03, 9.6117e-06, 7.2309e-03, 1.9626e-03,
         7.4503e-04, 4.3784e-04, 9.0166e-04, 2.4190e-05, 3.1094e-04, 2.0940e-03,
         5.4749e-03, 2.7977e-03, 1.2051e-04, 1.0583e-03, 2.8979e-04, 2.0454e-04,
         6.7734e-03, 1.2582e-03, 1.0795e-03, 2.2113e-04, 6.1929e-06, 8.9607e-05,
         2.9217e-03, 1.6285e-03, 9.9371e-04, 6.8037e-05, 1.5405e-03, 8.1048e-03,
         1.3781e-03, 3.9617e-05, 2.4235e-03, 2.4337e-03, 1.0152e-03, 1.7398e-03,
         6.5376e-04, 1.0875e-04, 1.3650e-05, 4.9399e-04, 8.3091e-04, 3.2598e-04,
         1.5872e-03, 1.0546e-03, 2.3059e-03, 5.9109e-06, 2.1607e-03, 2.2526e-04,
         2.5568e-03, 3.2081e-04, 4.6027e-03, 5.6956e-03, 1.3055e-03, 3.6201e-05,
         1.5651e-02, 1.1517e-03, 6.3532e-04, 4.6747e-03, 9.3853e-05, 1.3051e-05,
         3.6102e-04, 3.9944e-06, 5.7350e-05, 6.5346e-04, 1.6668e-04, 2.7213e-03,
         2.5723e-04, 1.9357e-04, 1.4167e-03, 3.0762e-03, 3.7520e-03, 3.3558e-03,
         8.1116e-03, 1.3999e

In [116]:
s[0]

tensor([3.8418e-03, 4.2544e-03, 1.4207e-03, 9.6117e-06, 7.2309e-03, 1.9626e-03,
        7.4503e-04, 4.3784e-04, 9.0166e-04, 2.4190e-05, 3.1094e-04, 2.0940e-03,
        5.4749e-03, 2.7977e-03, 1.2051e-04, 1.0583e-03, 2.8979e-04, 2.0454e-04,
        6.7734e-03, 1.2582e-03, 1.0795e-03, 2.2113e-04, 6.1929e-06, 8.9607e-05,
        2.9217e-03, 1.6285e-03, 9.9371e-04, 6.8037e-05, 1.5405e-03, 8.1048e-03,
        1.3781e-03, 3.9617e-05, 2.4235e-03, 2.4337e-03, 1.0152e-03, 1.7398e-03,
        6.5376e-04, 1.0875e-04, 1.3650e-05, 4.9399e-04, 8.3091e-04, 3.2598e-04,
        1.5872e-03, 1.0546e-03, 2.3059e-03, 5.9109e-06, 2.1607e-03, 2.2526e-04,
        2.5568e-03, 3.2081e-04, 4.6027e-03, 5.6956e-03, 1.3055e-03, 3.6201e-05,
        1.5651e-02, 1.1517e-03, 6.3532e-04, 4.6747e-03, 9.3853e-05, 1.3051e-05,
        3.6102e-04, 3.9944e-06, 5.7350e-05, 6.5346e-04, 1.6668e-04, 2.7213e-03,
        2.5723e-04, 1.9357e-04, 1.4167e-03, 3.0762e-03, 3.7520e-03, 3.3558e-03,
        8.1116e-03, 1.3999e-03, 1.3684e-

In [47]:
s.detach().cpu().numpy()

array([[1.43374987e-02, 2.98864616e-04, 5.00092469e-03, 4.33686702e-03,
        2.23848590e-04, 1.04449740e-04, 1.88874674e-03, 4.95864369e-04,
        1.20360858e-03, 2.41601164e-03, 1.57194945e-03, 1.47000837e-05,
        6.53460808e-03, 1.84603559e-03, 1.72503281e-03, 1.76809583e-04,
        7.41537544e-04, 1.12845551e-03, 1.33090348e-08, 1.87009026e-03,
        3.71120659e-05, 6.03178516e-03, 8.57050938e-04, 9.22304116e-06,
        2.62246421e-03, 1.71023968e-03, 1.70792954e-03, 1.66873459e-03,
        6.98595075e-04, 4.69151698e-03, 1.27028511e-03, 1.97395988e-04,
        1.02537032e-03, 1.32954435e-03, 1.08018972e-03, 3.49264126e-03,
        1.22834696e-04, 1.94777704e-06, 1.49804409e-05, 2.77141839e-06,
        7.47818383e-04, 1.61630800e-03, 1.34545201e-06, 8.84008884e-08,
        2.30433978e-03, 3.03325779e-03, 3.51930992e-03, 1.78184109e-05,
        5.31370798e-03, 5.63195408e-06, 2.28732912e-04, 9.26771099e-05,
        1.50153541e-03, 5.28894505e-03, 4.83373227e-03, 1.244709

In [75]:
o

tensor([[[-0.1758],
         [-0.2710],
         [-0.2833],
         [-0.1896],
         [-0.1712],
         [-0.1583],
         [-0.1247],
         [-0.1774],
         [-0.2135],
         [-0.1891],
         [-0.1986],
         [-0.2082],
         [-0.1938],
         [-0.1855],
         [-0.1563],
         [-0.1637],
         [-0.1945],
         [-0.1556],
         [-0.1158],
         [-0.1527],
         [-0.2552],
         [-0.1858],
         [-0.2097],
         [-0.1977],
         [-0.1731],
         [-0.1450],
         [-0.1488],
         [-0.1957],
         [-0.1519],
         [-0.1670],
         [-0.1999],
         [-0.2151],
         [-0.2092],
         [-0.1931],
         [-0.1988],
         [-0.1926],
         [-0.1860],
         [-0.2129],
         [-0.1787],
         [-0.1193],
         [-0.1657],
         [-0.1299],
         [-0.1798],
         [-0.1518],
         [-0.2064],
         [-0.0946],
         [-0.1615],
         [-0.1530],
         [-0.2240],
         [-0.1860],


In [82]:
v = o.cpu().numpy()

In [138]:
v[0][100-83:].shape

(83, 1)

In [87]:
d['recon'] = v[0]
d

,NAME,Cumul_precipitation,date,recon
0,Bobo-Dioulasso,0.0,1/1/2019 0:00,-0.175764
1,Bobo-Dioulasso,0.0,1/2/2019 0:00,-0.270958
2,Bobo-Dioulasso,0.0,1/3/2019 0:00,-0.283332
3,Bobo-Dioulasso,0.0,1/4/2019 0:00,-0.189613
4,Bobo-Dioulasso,0.0,1/5/2019 0:00,-0.171197
...,...,...,...,...
95,Bobo-Dioulasso,0.0,4/6/2019 0:00,-0.177099
96,Bobo-Dioulasso,0.0,4/7/2019 0:00,-0.131717
97,Bobo-Dioulasso,0.0,4/8/2019 0:00,-0.181124
98,Bobo-Dioulasso,0.0,4/9/2019 0:00,-0.224927


In [97]:
d['loss'] = s[0]
d

,NAME,Cumul_precipitation,date,recon,loss
0,Bobo-Dioulasso,0.0,1/1/2019 0:00,-0.175764,0.010006
1,Bobo-Dioulasso,0.0,1/2/2019 0:00,-0.270958,0.029886
2,Bobo-Dioulasso,0.0,1/3/2019 0:00,-0.283332,0.001126
3,Bobo-Dioulasso,0.0,1/4/2019 0:00,-0.189613,0.004613
4,Bobo-Dioulasso,0.0,1/5/2019 0:00,-0.171197,0.002279
...,...,...,...,...,...
95,Bobo-Dioulasso,0.0,4/6/2019 0:00,-0.177099,0.000491
96,Bobo-Dioulasso,0.0,4/7/2019 0:00,-0.131717,0.005500
97,Bobo-Dioulasso,0.0,4/8/2019 0:00,-0.181124,0.001907
98,Bobo-Dioulasso,0.0,4/9/2019 0:00,-0.224927,0.004004


In [201]:
result = predict_dataframe(args, df_raw, 'Cumul_precipitation', model, 0.44)

Processing batch 1/218
Processing batch 2/218
Processing batch 3/218
Processing batch 4/218
Processing batch 5/218
Processing batch 6/218
Processing batch 7/218
Processing batch 8/218
Processing batch 9/218
Processing batch 10/218
Processing batch 11/218
Processing batch 12/218
Processing batch 13/218
Processing batch 14/218
Processing batch 15/218
Processing batch 16/218
Processing batch 17/218
Processing batch 18/218
Processing batch 19/218
Processing batch 20/218
Processing batch 21/218
Processing batch 22/218
Processing batch 23/218
Processing batch 24/218
Processing batch 25/218
Processing batch 26/218
Processing batch 27/218
Processing batch 28/218
Processing batch 29/218
Processing batch 30/218
Processing batch 31/218
Processing batch 32/218
Processing batch 33/218
Processing batch 34/218
Processing batch 35/218
Processing batch 36/218
Processing batch 37/218
Processing batch 38/218
Processing batch 39/218
Processing batch 40/218
Processing batch 41/218
Processing batch 42/218
P

In [202]:
result

,NAME,Cumul_precipitation,date,reconstruction_error,reconstructed,is_anomaly
0,Bobo-Dioulasso,0.0,1/1/2019 0:00,0.000287,-0.141817,False
1,Bobo-Dioulasso,0.0,1/2/2019 0:00,0.002062,-0.380131,False
2,Bobo-Dioulasso,0.0,1/3/2019 0:00,0.000341,-0.154620,False
3,Bobo-Dioulasso,0.0,1/4/2019 0:00,0.000363,-0.159593,False
4,Bobo-Dioulasso,0.0,1/5/2019 0:00,0.000216,-0.122972,False
...,...,...,...,...,...,...
21778,Po,0.0,4/26/2025 0:00,0.006003,-0.648586,False
21779,Po,10.7,4/27/2025 0:00,0.018649,9.556810,False
21780,Po,0.0,4/28/2025 0:00,0.008873,0.788556,False
21781,Po,0.0,4/29/2025 0:00,0.002776,-0.441066,False


In [205]:
result.to_csv(result, 'result.csv', index=False)

TypeError: argument of type 'method' is not iterable

In [199]:
b

array([[-0.0355285 ],
       [-0.14645226],
       [-0.13800223],
       ...,
       [ 0.20297949],
       [-0.12076529],
       [10.012269  ]], dtype=float32)